In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
gamma_data_path = "../data/processed/magic04.data"
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
gamma_data = pd.read_csv(gamma_data_path, names=cols)
gamma_data.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
gamma_data["class"].unique()

array(['g', 'h'], dtype=object)

In [4]:
gamma_data["class"] = (gamma_data["class"] == "g").astype(int)
print("Making class into a binary variable")

Making class into a binary variable


In [5]:
gamma_data["class"].unique()

array([1, 0])

In [ ]:
for label in cols[:-1]:
  plt.hist(gamma_data[gamma_data["class"]==1][label], color='blue', label='gamma', alpha=0.7, density=True)
  plt.hist(gamma_data[gamma_data["class"]==0][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

In [10]:

train, valid, test = np.split(gamma_data.sample(frac=1), [int(.6*len(gamma_data)), int(.8*len(gamma_data))])

In [8]:
def scale_dataset(dataframe, oversample=False):
  features = dataframe[dataframe.columns[:-1]].values
  label = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  features = scaler.fit_transform(features)

  if oversample:
    ros = RandomOverSampler()
    features, label = ros.fit_resample(features, label)

  data = np.hstack((features, np.reshape(label, (-1, 1))))

  return data, features, label

In [9]:
# X means features, y means label
train, X_train, y_train = scale_dataset(train, oversample=True)
# dataframe train doesnt get used a
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

# KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [36]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [39]:
y_pred = knn_model.predict(X_test)

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73      1305
           1       0.86      0.86      0.86      2499

    accuracy                           0.81      3804
   macro avg       0.79      0.79      0.79      3804
weighted avg       0.81      0.81      0.81      3804



> ### precision
> - tp/tp+fp
> - The number of instances that are relevant, out of the total instances the model retrieved.
> - How many retrieved items are relevant?
> ### recall
> - tp/tp+fn
> - The number of instances which the model correctly identified as relevant out of the total relevant instances.
> - How many relevant items are retrieved?
- [precision and recall medium article](https://towardsdatascience.com/precision-and-recall-88a3776c8007)

# Naive Bayes

In [41]:
from sklearn.naive_bayes import GaussianNB

In [42]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train, y_train)

In [43]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.41      0.51      1305
           1       0.74      0.89      0.81      2499

    accuracy                           0.73      3804
   macro avg       0.70      0.65      0.66      3804
weighted avg       0.72      0.73      0.71      3804



### Naive Bayes
- [Navie Bayes explaination](https://www.youtube.com/watch?v=HZGCoVF3YvM)
- 

# Logistic Regression

In [45]:
from sklearn.linear_model import LogisticRegression

In [46]:
lg_model = LogisticRegression()
lg_model = lg_model.fit(X_train, y_train)

In [47]:
y_pred = lg_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.72      0.69      1305
           1       0.85      0.81      0.83      2499

    accuracy                           0.78      3804
   macro avg       0.76      0.77      0.76      3804
weighted avg       0.79      0.78      0.78      3804



### Logistic Regression
- logistic regression is a classification algorithm that tries to fit data to the sigmoid function.

# Support Vector Machine (SVM)

In [48]:
from sklearn.svm import SVC

In [49]:
svm_model = SVC()
svm_model = svm_model.fit(X_train, y_train)

In [50]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80      1305
           1       0.89      0.90      0.90      2499

    accuracy                           0.86      3804
   macro avg       0.85      0.84      0.85      3804
weighted avg       0.86      0.86      0.86      3804



### Support Vector Machine (SVM)
- SVM find the best hyperplane to separate the data

# Neural Network

In [11]:
import tensorflow as tf

In [12]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [15]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
      tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(num_nodes, activation='relu'),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr), loss='binary_crossentropy',
                  metrics=['accuracy'])
  history = nn_model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0
  )

  return nn_model, history

In [ ]:
least_val_loss = float('inf')
least_loss_model = None
epochs=100
for num_nodes in [16, 32, 64]:
  for dropout_prob in[0, 0.2]:
    for lr in [0.01, 0.005, 0.001]:
      for batch_size in [32, 64, 128]:
        print(f"{num_nodes} nodes, dropout {dropout_prob}, lr {lr}, batch size {batch_size}")
        model, history = train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs)
        plot_history(history)
        val_loss = model.evaluate(X_valid, y_valid)[0]
        if val_loss < least_val_loss:
          least_val_loss = val_loss
          least_loss_model = model

In [21]:
y_pred = least_loss_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).reshape(-1, 1)
print(classification_report(y_test, y_pred))

119/119 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.91      0.72      0.81      1402
           1       0.86      0.96      0.90      2402

    accuracy                           0.87      3804
   macro avg       0.88      0.84      0.86      3804
weighted avg       0.88      0.87      0.87      3804

